# Corrected Convex Nonparametric Least Squares (C2NLS)

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Date : June 17, 2020

References:

[1] Kuosmanen, T. and Johnson, A. L. (2010). Data envelopment analysis as nonparametric least-squares regression. Operations Research, 58, pp. 149–160.

# Example

### Stage 1: Esimate $E(y_i | x_i)$ by solving the CNLS problem.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import the package pystoned
from pystoned import *

In [3]:
# import Finnish electricity distribution firms data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED-Tutorials/master/Data/firms.csv'
df = pd.read_csv(url, error_bad_lines=False)
df.head(5)

,OPEX,CAPEX,TOTEX,Energy,Length,Customers,PerUndGr
0,681,729,1612,75,878,4933,0.11
1,559,673,1659,62,964,6149,0.21
2,836,851,1708,78,676,6098,0.75
3,7559,8384,18918,683,12522,55226,0.13
4,424,562,1167,27,697,1670,0.03


In [4]:
# output
y = df['Energy']

# inputs
x1 = df['OPEX']
x1 = np.asmatrix(x1).T
x2 = df['CAPEX']
x2 = np.asmatrix(x2).T
x = np.concatenate((x1, x2), axis=1)

In [5]:
# define and solve the CNLS model

cet = "addi"
fun = "prod"
rts = "vrs"

model = CNLS.cnls(y, x, cet, fun, rts)

# using local solver (MOSEK API)
from pyomo.opt import SolverFactory
opt = SolverFactory("mosek")
results = opt.solve(model, tee=True)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 7921            
  Cones                  : 0               
  Scalar variables       : 445             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.00    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 89
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0              

In [6]:
# retrive the alpha
val = list(model.a[:].value)
alpha = np.asarray(val)
alpha

array([-22.93574197, -22.9865429 , -22.86379597,  33.46107704,
       -22.93816626, -17.75390496, -22.92070101, -17.79487848,
       -22.90315617, -17.87076426,  89.09009201,  90.88462434,
       -22.83508621, -17.80922168, 102.38639157, -17.67072607,
       -22.96478157, -22.91711305, -22.90710605, -17.6820098 ,
       -17.28941355, -21.14677138, -25.26674503, -17.63199337,
       -22.95755108, -22.84459768, -22.84621257,  33.4613221 ,
       -22.96383052, -17.80276734, -22.90794557, 104.33191265,
       -17.7114322 , -17.77622109, -19.65343202, -17.66687285,
       -17.70397656, -22.96480511, -22.99005476, -17.6281484 ,
       -17.66150147, -22.99221792,  33.46178545, -22.92142768,
       -22.89371926, -22.94276172, -17.6369817 , -22.99131548,
       -17.66649268,  33.45230894, -22.94213913, -17.70744385,
       -22.87512746, -17.66156705, -22.91239992,  22.31227387,
       -14.38711198, -22.8824601 , -22.0951086 , -17.76763495,
       -24.54292696,  33.46143999, -17.62882348, -13.85

In [7]:
# retrive the residuals
val = list(model.e[:].value)
eps = np.asarray(val)
eps

array([  -2.80240049,    1.4139941 ,  -22.22370778, -350.91104218,
        -13.70003771,  101.01492222,  -28.87235177,  -14.03966648,
         -0.84743059,   56.89437246,  285.50684532,  679.38389548,
        -20.229963  ,  -70.00784192,   10.50653957,   74.59739943,
         -6.53896952,  -30.07641319,  -40.134715  ,  -27.77778312,
         48.75951015,   87.00804975,   22.48052139,   23.95079962,
         -2.14170243,  -22.83655374,  -37.86099929, -351.07100145,
         66.85755272,  -21.70272781,   -8.95127728,  216.0060913 ,
         37.59744429,  -31.81138603,  -23.78465848, -201.34117915,
        -69.29026659,    6.26873673,    3.75574364,  -74.60736509,
         -1.97475364,  -11.97099005, -236.74633737,    6.57459175,
         11.65690679,  -20.00411823,  -67.2145622 ,   -5.23924544,
        -55.94688679,  265.51405577,    1.92405574,  -17.65073349,
          4.65633465,   38.85438249,   -2.90330145,  349.52822197,
        163.99055389,   35.00139386,   28.39297571,  -99.13207

### Stage 2: Shift the residuals analogous to the COLS procedure.

In [8]:
# shift the residuals and constant term
res = CCNLS.ccnls(eps, alpha)

In [9]:
# adjusted residuals
eps_ccnls = res[0]
eps_ccnls

array([ -682.18629597,  -677.96990138,  -701.60760326, -1030.29493766,
        -693.08393319,  -578.36897326,  -708.25624725,  -693.42356196,
        -680.23132607,  -622.48952301,  -393.87705016,     0.        ,
        -699.61385848,  -749.39173739,  -668.87735591,  -604.78649605,
        -685.922865  ,  -709.46030867,  -719.51861048,  -707.16167859,
        -630.62438533,  -592.37584573,  -656.90337409,  -655.43309586,
        -681.52559791,  -702.22044922,  -717.24489477, -1030.45489693,
        -612.52634276,  -701.08662329,  -688.33517276,  -463.37780418,
        -641.78645119,  -711.19528151,  -703.16855396,  -880.72507463,
        -748.67416207,  -673.11515875,  -675.62815184,  -753.99126057,
        -681.35864912,  -691.35488553,  -916.13023285,  -672.80930373,
        -667.72698869,  -699.38801371,  -746.59845768,  -684.62314092,
        -735.33078227,  -413.86983971,  -677.45983974,  -697.03462897,
        -674.72756082,  -640.52951299,  -682.28719693,  -329.85567351,
      

In [10]:
# adjusted intercept
alpha_ccnls = res[1]
alpha_ccnls

array([656.44815351, 656.39735258, 656.52009951, 712.84497252,
       656.44572922, 661.62999052, 656.46319447, 661.589017  ,
       656.48073931, 661.51313122, 768.47398749, 770.26851982,
       656.54880926, 661.5746738 , 781.77028705, 661.71316941,
       656.41911391, 656.46678243, 656.47678943, 661.70188568,
       662.09448193, 658.2371241 , 654.11715045, 661.75190211,
       656.4263444 , 656.5392978 , 656.53768291, 712.84521758,
       656.42006496, 661.58112814, 656.4759499 , 783.71580813,
       661.67246328, 661.60767439, 659.73046346, 661.71702263,
       661.67991892, 656.41909037, 656.39384072, 661.75574708,
       661.72239401, 656.39167756, 712.84568093, 656.4624678 ,
       656.49017622, 656.44113376, 661.74691378, 656.39258   ,
       661.7174028 , 712.83620442, 656.44175635, 661.67645163,
       656.50876802, 661.72232843, 656.47149555, 701.69616935,
       664.9967835 , 656.50143538, 657.28878688, 661.61626053,
       654.84096852, 712.84533547, 661.755072  , 665.53